#1．Gdriveにマウント

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#2．ライブラリのインポート

In [0]:
#Loading dataset
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import itertools

#3．データの取り込み

In [0]:
path = "/content/gdrive/My Drive/Colab Notebooks/GCI2019/competition2/input/"

train = pd.read_csv(path +  'train.csv')
test = pd.read_csv(path +  'test.csv')

##3-1．欠損値の処理

In [0]:
density = pd.concat([train['density'], test['density']])
pH = pd.concat([train['pH'], test['pH']])

train['density'].fillna(density.mean(), inplace=True)
test['density'].fillna(density.mean(), inplace=True)

train['pH'].fillna(pH.mean(), inplace=True)
test['pH'].fillna(pH.mean(), inplace=True)

better_size:  0.1 better_score(mse):  0.32018608149050043 better_factor: ('fixed acidity', 'citric acid', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol')


#1.データの取り込み

In [0]:
#データの取り込み
df
df_test

#力任せ方式分析

In [0]:
# common
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
# SVM
from sklearn.svm import LinearSVC
# 決定木
from sklearn.tree import  DecisionTreeClassifier
# k-NN
from sklearn.neighbors import  KNeighborsClassifier

インデントの数＝モジュール数

In [0]:
def best_model(train, test, model_list, test_size):
  #初期値の設定
  best_score = 0
  best_size = 0
  best_factor = ""
  best_method = ""
  a_list=[] #説明変数のリスト
  #抽出するデータ項目の全組み合わせ
  for i in range(1, len(test.columns)+1):
    a = list(itertools.combinations(test.columns, i))
    #print(i, a)
    a_list.append(a)
  for i in a_list[::-1]: #iはn個抽出した時の項目の組み合わせを格納
    for j in i: #jはiの組み合わせを1組ずつ抽出
      #print('current check: ', j)
      #実際の抽出
      X = train[list(j)]
      y = train['quality']

      #test_size = [0.5, 0.3, 0.25, 0.2, 0.1]
      #n個の組み合わせ毎に一番良い値とその時のテストサイズを抽出するための箱
      better_score = 0 #n個の組み合わせの計算毎にリセットされる
      better_size = 0
      better_factor = ""
      better_method = ""
      for size in test_size:
        X_train, X_test, y_train, y_test = train_test_split(
              X, y, test_size=size, random_state=0)
        #試したいモデルを入れる

        for i in model_list:
            model = i
            model.fit(X_train, y_train)
            #n個の組み合わせ毎に一番良い値とその時のテストサイズを実際に抽出
            if better_score == 0: #1番はじめ
                better_score = model.score(X_test, y_test)
                better_size = size
                better_factor = j              
                better_method = model.__class__.__name__
            elif model.score(X_test, y_test) > better_score:
                better_score = model.score(X_test, y_test)
                better_size = size
                better_factor = j    
                better_method = model.__class__.__name__
      print('better_method: {}'.format(better_method), 'better_size: ', better_size, 'better_score(acc): ', better_score, 'better_factor: {}{}'.format(len(better_factor), better_factor))
      #n個の組み合わせの全ての計算が終わったあとに、bestなものを選ぶ
      if best_score == 0:
          best_score = better_score
          best_size = better_size
          best_factor = better_factor
          best_method = better_method
      if better_score > best_score:
          best_score = better_score
          best_size = better_size
          best_factor = better_factor
          best_method = better_method    #n個の組み合わせ毎の計算が終わったら、最善の結果を出力する
  print('best_method: {}'.format(best_method),'best_size: ', best_size, 'best_score(acc): ', best_score, 'best_factor: {}'.format(best_factor))
  return best_method, best_size, best_score, best_factor

In [0]:
test_size = [0.5, 0.3, 0.25, 0.2, 0.1]

In [0]:
model_list = [RandomForestRegressor(random_state=42),LinearRegression()]

In [11]:
best_model2(train, test, model_list, test_size)

UnboundLocalError: ignored

#2.説明変数の総当りリスト

In [0]:
def X_arr(train, test): #説明変数の総当りリスト
    import numpy as np
    #説明変数のリストを作って、説明変数を抽出
    X_list = []
    #説明変数の抽出
    for i in range(1, len(test.columns)+1):
      X = list(itertools.combinations(test.columns, i))
      for a in X:
        X_list.append(a)
    X_arr = np.array(X_list)
    return X_arr

#3.疑似訓練データと疑似テストデータの分類

In [0]:
#疑似訓練データと疑似テストデータの分類
for size in test_size:
  #この辺にgridsearch
  X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=size, random_state=0)

In [0]:
#モデルの作成=>自分で作ったほうが良い
def make_models(a, b):
    models = []
    models.append(a, b) #ここでエラー
    return models

#4.モデルリストの作成

In [0]:
models = [RandomForestRegressor(random_state=42),LinearRegression()]

#5.訓練から評価までは一括でやる

説明変数の数とデータサイズの2段階

In [0]:
a = 0 #best_scoreとbest_methodを判断するために過学習の小さいものから選ぶ
for i in models:
    model = i
    model.fit(X_train, y_train)

In [0]:
for model in models

In [0]:
#予測
def pred(models):
    return pred

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
pred_test = model.predict(X_test)

In [0]:
#評価
def eval(pred):
    return R1, R2, mse, 
  if better_score == 0:
    better_score = model.score(X_test, y_test)
    better_size = size
    better_factor = j
  elif model.score(X_test, y_test) > better_score:
    better_score = model.score(X_test, y_test)
    better_size = size
    better_factor = j
print('better_size: ', better_size, 'better_score(mse): ', better_score, 'better_factor: {}'.format(j))
if best_score == 0:
    best_score = better_score
    best_size = better_size
    best_factor = better_factor
if better_score > best_score:
    best_score = better_score
    best_size = better_size
    best_factor = better_factor 

In [0]:
for i in a_list[::-1]: #iはn個抽出した時の項目の組み合わせを格納
  for j in i: #jはiの組み合わせを1組ずつ抽出
    #print('current check: ', j)
    #実際の抽出
    X = train[list(j)]
    y = train['quality']

    test_size = [0.5, 0.3, 0.25, 0.2, 0.1]
    #n個の組み合わせ毎に一番良い値とその時のテストサイズを抽出
    better_score = 0 #n個の組み合わせの計算毎にリセットされる
    better_size = 0
    better_factor = ""
    for size in test_size:
      #この辺にgridsearch
      X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=size, random_state=0)
      model = RandomForestRegressor(random_state=42)
      model.fit(X_train, y_train)
      pred_test = model.predict(X_test)
      #print(size, j)
      if better_score == 0:
        better_score = model.score(X_test, y_test)
        better_size = size
        better_factor = j
      elif model.score(X_test, y_test) > better_score:
        better_score = model.score(X_test, y_test)
        better_size = size
        better_factor = j
    print('better_size: ', better_size, 'better_score(mse): ', better_score, 'better_factor: {}'.format(j))
    if best_score == 0:
        best_score = better_score
        best_size = better_size
        best_factor = better_factor
    if better_score > best_score:
        best_score = better_score
        best_size = better_size
        best_factor = better_factor 

#練習

In [0]:
model_list=[
            RandomForestRegressor(random_state=42),
            LinearRegression(),
            ]
a = 0 #best_scoreとbest_methodを判断するために過学習の小さいものから選ぶ
for i in model_list:
    model = i
    model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False) 0.11134050321938538
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 0.12679157902166016


In [0]:
# working place. everything 
def homework(X_train, X_test, y_train, y_test,best_score,best_method):
    model_list=[
                RandomForestRegressor(random_state=42),
                LinearRegression(),
                ]
    a = 0 #best_scoreとbest_methodを判断するために過学習の小さいものから選ぶ
    for i in model_list:
        model = i
        model.fit(X_train, y_train)
        pirnt(model, model.score(X_test, y_test))
        
        D = abs(model.score(X_train, y_train) - model.score(X_test, y_test))
        if best_score == 0: #1番はじめ
            best_score = model.score(X_test, y_test)
            best_method = model.__class__.__name__
            a = D
        elif model.score(X_test, y_test) > best_score:
            best_score = model.score(X_test, y_test)
            best_method = model.__class__.__name__
            a = D
        elif model.score(X_test, y_test) == best_score:
            if D < a:
              a = D
              best_score = model.score(X_test, y_test)
              best_method = model.__class__.__name__
    my_result = best_method
    return my_result